In [44]:
import mujoco_py
import os
import numpy as np

In [114]:
xml_path = os.path.join(os.getcwd(), 'xmls', 'inverted_pendulum.xml')
model = mujoco_py.load_model_from_path(xml_path)
sim = mujoco_py.MjSim(model)

In [115]:
sim.data.qpos[:] = [0.00184113, -0.00434254]
sim.data.qvel[:] = [0.18378366, -0.43146432]
print(sim.data.qpos, sim.data.qvel)
sim.data.ctrl[:] = [1.0]
sim.step()
print(sim.data.qpos, sim.data.qvel)

[ 0.00184113 -0.00434254] [ 0.18378366 -0.43146432]
[ 0.00734021 -0.01716705] [ 0.36588873 -0.8491507 ]


In [60]:
print(sim.data.qpos, sim.data.qvel)
sim.data.ctrl[:] = [1.000001]
sim.step()
print(sim.data.qpos, sim.data.qvel)

[ 0.02920693 -0.06747734] [ 0.72686904 -1.66372661]
[ 0.04553858 -0.10479942] [ 0.90616025 -2.06864007]


In [84]:
print(sim.model.opt.timestep)
print(sim.model.actuator_ctrlrange)

0.02
[[-3.  3.]]


In [107]:
print(np.concatenate([sim.data.qpos, sim.data.qvel]))
print(sim.model.nq, sim.model.nv)

[0. 0. 0. 0.]
2 2


[0.  0.  1.4 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0. ] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[-1.19992859e-06 -1.87591264e-09  1.39995559e+00  1.00000000e+00
 -3.80958646e-09  6.23973182e-06 -3.53397095e-08 -3.99644872e-07
  2.52122532e-05  8.11687406e-09  9.12887865e-09  1.42592981e-05
 -1.95591133e-04 -3.54987016e-04 -6.99724215e-09  1.36042558e-05
 -1.95247821e-04 -3.55003079e-04 -8.27498988e-06  1.01793273e-05
 -1.73780817e-06  8.38457556e-06 -1.00221171e-05 -1.63270040e-06] [-9.60139621e-04 -1.14466936e-06 -2.96018504e-02 -4.40200557e-06
  9.46294582e-03 -4.56441332e-05 -2.30146924e-04  1.12503527e-02
  4.71852751e-06  4.40376581e-06  7.94610868e-03 -1.19865525e-01
 -2.24645671e-01 -4.94740417e-06  7.60817568e-03 -1.19662689e-01
 -2.24661581e-01 -6.18482314e-03  7.76909924e-03 -1.53741266e-03
  6.25402037e-03 -7.66784791e-03 -1.47044881e-03]
